In [1]:
from fastapi import FastAPI

In [2]:
from fastapi.openapi.docs import get_swagger_ui_html

In [3]:
import secrets

from fastapi import Depends, FastAPI, HTTPException, status
from fastapi.security import HTTPBasic, HTTPBasicCredentials

app = FastAPI(docs_url=None, redoc_url=None)

security = HTTPBasic()


def get_current_username(credentials: HTTPBasicCredentials = Depends(security)):
    correct_username = secrets.compare_digest(credentials.username, "user")
    correct_password = secrets.compare_digest(credentials.password, "password")
    if not (correct_username and correct_password):
        raise HTTPException(
            status_code=status.HTTP_401_UNAUTHORIZED,
            detail="Incorrect email or password",
            headers={"WWW-Authenticate": "Basic"},
        )
    return credentials.username


@app.get("/users/me")
def read_current_user(username: str = Depends(get_current_username)):
    return {"username": username}

In [4]:
@app.get("/docs")
async def get_documentation(username: str = Depends(get_current_username)):
    return get_swagger_ui_html(openapi_url="/openapi.json", title="docs")

In [ ]:
if __name__ == "__main__":
    import nest_asyncio
    nest_asyncio.apply()
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8080)

INFO:     Started server process [4881]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8080 (Press CTRL+C to quit)


INFO:     127.0.0.1:57320 - "GET /docs HTTP/1.1" 401 Unauthorized
INFO:     127.0.0.1:57320 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:57320 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:57568 - "GET /docs HTTP/1.1" 401 Unauthorized
INFO:     127.0.0.1:57568 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:57569 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:57569 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:57569 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:57568 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:57568 - "GET /openapi.json HTTP/1.1" 200 OK
